In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from astropy.io import fits
from astropy import wcs
from astropy.table import Table
from matplotlib.gridspec import GridSpec
import pandas as pd
from pandas import DataFrame
from astropy.cosmology import Planck15 as cosmo
import astropy.units as u
import ast
from astropy.visualization import quantity_support

%matplotlib inline

In [8]:
table_path = '../tables/'
cat_path = '../CANDLES_data/goods_catalogs/'
total_path = '../total_gal/'
cutout_path = '../CANDLES_data/total_cutouts/'
save_cutout_path = '../CANDLES_data/merger_cutouts/'
seg_path = '../CANDLES_data/goodsn_3dhst_v4.0_f160w/'
Ha_path = '../merger_candidates/'

In [9]:
def Scaling_factor(X, m, M):              #X = Counts, m = Vmin, M = Vmax
    m_mask = np.zeros_like(X)
    M_mask = np.zeros_like(X)

    for i in range(len(X)):
        for ii in range(len(X[0])):
            if X[i][ii] <= m:
                m_mask[i][ii] = 1 
            
            if X[i][ii] >= M:
                M_mask[i][ii] = 1 
             
    scl_img =  np.arcsinh(X - m)/np.arcsinh(M - m)
            
    for i in range(len(X)):
        for ii in range(len(X[0])):
            if m_mask[i][ii] == 1:
                scl_img[i][ii] = 0
            if M_mask[i][ii] == 1:
                scl_img[i][ii] = 1
            
    return scl_img

Attempting to incorporate segmentation maps

In [24]:
seg_N = fits.open(seg_path+'goodsn_3dhst.v4.0.F160W_seg.fits')[0].data

field = 'GN1'
Subjects = [37102,37836,37193,37636,36420,36795,37271,37738,37957,36514,36808,37343,37750,37363,
            37786,37996,36928,37395,37787,38027,36651,37406,38041,36717,37006,37424,37807,37067,
            37810,36736,37074,37615,37821]

rows = []
cols = []
Img_Ha = []
Img_Orig = []

#Opening the Fits Files for every ID
for i in Subjects:
    Ha = fits.open(Ha_path+'{}/{}_{}.full.fits'.format(field,field,i))
    Line = Ha['Line','Ha'].data      #Halpha data
    Continuum = Ha['Continuum','Ha'].data
    Contam = Ha['Contam','Ha'].data
    Raw = Ha['DSCI','F105W'].data   #Original Image
    
    Img_Ha.append(Line - Continuum - Contam)  #Fixing the Errors on Halpha
    Img_Orig.append(Raw)
    
for j in range(len(Img_Ha)):
#Deciding the ID and Observation Space
    #j = 0
    Halph = Img_Ha[j][19:60,19:60]
    Original = Img_Orig[j][19:60,19:60]            

idx = 0

for g in (Subjects):
    locations= np.argwhere(seg_N == g).T
    rows.extend(locations[0])
    cols.extend(locations[1])

    rowmin = min(rows)
    rowmax = max(rows)
    colmin = min(cols)
    colmax = max(cols)
    segsize = seg_N[rowmin : rowmax+1, colmin : colmax+1]
    newseg = np.zeros_like(segsize)

    for j in range(len(segsize)):
        for k in range(len(segsize[j])):
            newseg[j][k] = 1